In [2]:
import sys
import re
from enum import Enum
from dataclasses import dataclass
from typing import List
import heapq

In [3]:
class CellType(Enum):
    EMPTY = 0
    BLOCK = 1
    SOURCE = 2
    TARGET = 3
    VIA = 4

@dataclass
class Cell:
    kind: CellType
    cost: int = 0

@dataclass
class Coord:
    row: int
    col: int
    layer: int

In [7]:
class MazeRouter:
    NUM_LAYERS = 2

    def __init__(self,
                 in_path: str,
                 out_path: str, #Check if needed in future
                 routing: str = "routing.txt",
                 dimensions: str = "dimensions.txt"):
        self.in_path = in_path
        self.out_path = out_path
        self.routing = routing
        self.dimensions = dimensions

        self.cols = 0
        self.rows = 0
        self.Direction = 0
        self.via = 0
        self.via_count = 0 #Change: Number of times we change between layers.

        self.grid: List[List[List[Cell]]] = []
        self.nets: List[List[Coord]] = []
        self.same_layer_nets: List[List[Coord]] = []
        self.multi_layer_nets: List[List[Coord]] = []

        self.clear_routings()
        self.load_config()
        self.save_dimensions()

    def clear_routings(self):
        with open(self.routing, 'w'):
            pass
        print("clear_routing Successful")

    def load_config(self):
        with open(self.in_path, 'r') as f:
            header = f.readline().strip()
            c, r, b, v = map(int, header.split(','))
            self.cols, self.rows = c, r
            self.Direction, self.via = b, v

            self.grid = [
                [
                    [Cell(CellType.EMPTY) for j in range(self.cols)]
                    for i in range(self.rows)
                ]
                for layer_idx in range(self.NUM_LAYERS)
            ]

            for line in f:
                line = line.strip()
                if line.startswith("OBS"):
                    self.add_obstacle(line)
                elif line.startswith("net"):
                    self.add_net(line)

    def save_dimensions(self):
        with open(self.dimensions, 'w') as f:
            f.write(f"{self.cols} {self.rows}\n")

    def add_obstacle(self, text: str):
        m = re.match(r"OBS\s*\((\d+),\s*(\d+),\s*(\d+)\)", text)
        if not m:
            print("Incorrect Format.", file=sys.stderr)
            return
        layer, col, row = map(int, m.groups())
        if 0 <= layer < self.NUM_LAYERS and 0 <= row < self.rows and 0 <= col < self.cols:
            self.grid[layer][row][col].kind = CellType.BLOCK
        else:
            print("Obstacle out of Range", file=sys.stderr)

    def add_net(self, text: str):
        path: List[Coord] = []
        for m in re.finditer(r"\((\d+),\s*(\d+),\s*(\d+)\)", text):
            layer, col, row = map(int, m.groups())
            if 0 <= layer < self.NUM_LAYERS and 0 <= row < self.rows and 0 <= col < self.cols:
                path.append(Coord(row, col, layer))
            else:
                print("Coordinates out of Range", file=sys.stderr)
                return
        self.nets.append(path)

    def set_SourceAndTarget(self, net_coordinates: List[Coord]):
        for idx, coord in enumerate(net_coordinates):
            kind = CellType.SOURCE if idx == 0 else CellType.TARGET
            self.grid[coord.layer][coord.row][coord.col].kind = kind

    def categorize_nets(self):
        self.same_layer_nets.clear()
        self.multi_layer_nets.clear()
        for idx, net in enumerate(self.nets):
            layers = {coord.layer for coord in net}
            if len(layers) == 1:
                self.same_layer_nets.append(net)
                print(f"Net {idx} → SAME layer {layers.pop()}")
            else:
                self.multi_layer_nets.append(net)
                print(f"Net {idx} → MULTI layers {layers}")

    def back_propagate(self, target: Coord) -> List[Coord]:
        route: List[Coord] = [target]
        current = Coord(target.row, target.col, target.layer)
        via_used = False
        max_iters = self.rows * self.cols * 2
        iters = 0
        while self.grid[current.layer][current.row][current.col].kind != CellType.SOURCE:
            if iters > max_iters:
                print("aborting", file=sys.stderr)
                break
            iters += 1
            cell = self.grid[current.layer][current.row][current.col]
            if cell.kind == CellType.VIA and not via_used:
                via_used = True
                cell.kind = CellType.EMPTY
                current = Coord(current.row, current.col, (current.layer + 1) % self.NUM_LAYERS)
                route.insert(0, current)
                self.via_count += 1
                continue
            best_cost = float('inf')
            best_dir = None
            for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                nr, nc = current.row + dr, current.col + dc
                if not (0 <= nr < self.rows and 0 <= nc < self.cols):
                    continue
                neigh = self.grid[current.layer][nr][nc]
                if neigh.kind == CellType.SOURCE:
                    current = Coord(nr, nc, current.layer)
                    route.insert(0, current)
                    return route
                if neigh.kind not in (CellType.BLOCK, CellType.SOURCE) and neigh.cost > 0:
                    if neigh.cost < best_cost:
                        best_cost = neigh.cost
                        best_dir = (nr, nc)
            if best_dir:
                current = Coord(best_dir[0], best_dir[1], current.layer)
                route.insert(0, current)
            else:
                break
        return route

    def route_same_layer_all(self): # Change: Now routes all nets in the same layer, instead of just the first one.
        self.categorize_nets()
        with open(self.routing, 'a') as rout:
            for layer in range(self.NUM_LAYERS):
                layer_nets = [
                    net for net in self.same_layer_nets
                    if all(coord.layer == layer for coord in net)
                ]
                for idx, net in enumerate(layer_nets):
                    src, tgt = net[0], net[1]
                    if not self._lee_wave(src, tgt, layer):
                        print(f"Failed to route net{idx} on layer {layer}", file=sys.stderr)
                        continue
                    path = self.back_propagate(tgt)
                    rout.write(
                        f"net{idx}_L{layer} " +
                        " ".join(f"({coord.layer},{coord.col},{coord.row})" for coord in path) +
                        "\n"
                    )
                    for coord in path:
                        self.grid[coord.layer][coord.row][coord.col].kind = CellType.BLOCK

    def _lee_wave(self, src: Coord, tgt: Coord, L: int) -> bool: #Change: Now chooses which layer to flood, instaead of it being always L=0
        INF = float('inf')
        for row_idx in range(self.rows):
            for col_idx in range(self.cols): #Change: Clearing the cost on whiciever layer we are routing
                cell = self.grid[L][row_idx][col_idx]
                cell.cost = -1 if cell.kind == CellType.BLOCK else INF
        self.grid[L][src.row][src.col].cost = 0
        pq = [(0, src.row, src.col)]
        while pq: #Change:  same 2D Lee expansion, but to whichever layer index passed.
            acc, row_idx, col_idx = heapq.heappop(pq)
            if acc > self.grid[L][row_idx][col_idx].cost:
                continue
            if (row_idx, col_idx) == (tgt.row, tgt.col):
                return True
            for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                nr, nc = row_idx + dr, col_idx + dc
                if not (0 <= nr < self.rows and 0 <= nc < self.cols):
                    continue
                neigh = self.grid[L][nr][nc]
                if neigh.kind == CellType.BLOCK:
                    continue
                step = 1 if dr == 0 else self.Direction
                new_cost = acc + step
                if new_cost < neigh.cost:
                    neigh.cost = new_cost
                    heapq.heappush(pq, (new_cost, nr, nc))
        return False


In [9]:
def main():
    router = MazeRouter('input2.txt', 'output2.txt')

    print(f"Grid: {router.cols} cols × {router.rows} rows, "
          f"horizontal cost={router.Direction}, via cost={router.via}\n")

    print("Obstacles:")
    for layer in range(router.NUM_LAYERS):
        for r in range(router.rows):
            for c in range(router.cols):
                if router.grid[layer][r][c].kind == CellType.BLOCK:
                    print(f"  layer {layer}, row {r}, col {c}")
    print()

    print("Loaded nets:")
    for idx, net in enumerate(router.nets):
        pts = [(c.layer, c.row, c.col) for c in net]
        print(f"  net{idx}: {pts}")
    print()

    for net in router.nets:
        router.set_SourceAndTarget(net)

    router.categorize_nets()

    def fmt(nets):
        return [
            [(c.layer, c.row, c.col) for c in net]
            for net in nets
        ]

    print("Same‐layer nets:", fmt(router.same_layer_nets))
    print("Multi‐layer nets:", fmt(router.multi_layer_nets))
    print("\n=== ROUTING RESULT ===")
    with open(router.routing, 'r') as f:
        for line in f:
            print(line.strip())

    if router.same_layer_nets:
        src, tgt = router.same_layer_nets[0][0], router.same_layer_nets[0][1]
        if router._lee_wave(src, tgt, L=1):
            path = router.back_propagate(tgt)
            coords = [(c.layer, c.row, c.col) for c in path]
            print("\ncoords for net:", coords)

if __name__ == "__main__":
    main()


clear_routing Successful
Grid: 8 cols × 8 rows, horizontal cost=5, via cost=10

Obstacles:
  layer 0, row 2, col 0
  layer 0, row 2, col 1
  layer 0, row 2, col 2
  layer 0, row 2, col 3
  layer 0, row 3, col 3
  layer 0, row 4, col 3
  layer 1, row 2, col 0
  layer 1, row 2, col 1
  layer 1, row 2, col 2
  layer 1, row 2, col 3
  layer 1, row 3, col 6
  layer 1, row 4, col 6
  layer 1, row 5, col 6

Loaded nets:
  net0: [(1, 1, 0), (1, 3, 1)]

Net 0 → SAME layer 1
Same‐layer nets: [[(1, 1, 0), (1, 3, 1)]]
Multi‐layer nets: []

=== ROUTING RESULT ===

coords for net: [(1, 1, 0), (1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 1, 4), (1, 2, 4), (1, 3, 4), (1, 3, 3), (1, 3, 2), (1, 3, 1)]
